In [1]:
#Import libraries
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [2]:
#Read data
datafile = "climate.data"
f = open(datafile)
data = []
i = 0
l = f.readline()

while(l != ''):
    a = l.split()
    l2 = []
    for j in range(0,len(a),1):
        l2.append(float(a[j]))
    data.append(l2)
    l = f.readline()
f.close()

In [3]:
#Calculate number of rows read from file and the number of columns 
rows = len(data)
cols = len(data[0])
print(rows)
print(cols)

540
18


In [4]:
#Read labels
labelfile = "climate.trainlabels.0"
f = open(labelfile)
trainlabels = {}
n = []
n.append(0)
n.append(0)
l = f.readline()
while(l != ''):
    a = l.split()
    trainlabels[int(a[1])] = int(a[0])
    if(trainlabels[int(a[1])] == 0):
        trainlabels[int(a[1])] = -1
    l = f.readline()
f.close()

In [5]:
len(trainlabels)

485

In [6]:
def gini_calculation(data,trainlabels):
    """
    Calculate the best split for a dataset using the Gini index.

    Parameters:
    data (list of list of float): The dataset to split.
    trainlabels (dict): Dictionary of labels where keys are row indices and values are labels (-1 or 1).

    Returns:
    tuple: The column index and the split value for the best split.
    """
    
    rows = len(data)
    cols = len(data[0])
    
    # Initialize a list to hold the minimum Gini index and its corresponding split index for each column
    gini_indices = [[0,0] for _ in range(cols)]
    column = 0 # The column to split on
    split = 0 # The value to split on
    mingini = float("inf") # Minimum Gini index
    
    for j in range(cols):
        # Get the current column values
        col_current = [item[j] for item in data]
        
        # Sort the indices of the column based on its values
        indices = sorted(range(len(col_current)), key=lambda k: col_current[k])
        
        # Sort the column values
        col_sorted = sorted(col_current)
        
        # Calculate Gini index for each possible split point in the column
        gini_rows = []
        for i in range(1, rows):
            lsize = i
            rsize = rows - i
            lp = sum(1 for k in range(i) if trainlabels.get(indices[k]) == -1)
            rp = sum(1 for k in range(i, rows) if trainlabels.get(indices[k]) == -1)
            
            # Calculate the Gini index for this split
            gini = (lsize/rows) * (lp/lsize) * (1 - lp/lsize) + (rsize/rows) * (rp/rsize) * (1 - rp/rsize)
            gini_rows.append(gini)
            
        # Find the minimum Gini index in the current column
        mingini = min(gini_rows)
        mingini_indices = gini_rows.index(mingini) + 1
        
        # Store the minimum Gini index and its split index
        gini_indices[j][0] = mingini
        gini_indices[j][1] = mingini_indices
        
    #Find the column with minimum Gini index
    temp = gini_indices[0][0]
    for j in range(cols):
        if gini_indices[j][0] <= temp:
            temp = gini_indices[j][0]
            column = j
            split = gini_indices[j][1]
            if split != 0:
                split = (col_sorted[split] + col_sorted[split-1])/2
    return column, split

In [7]:
column, split = gini_calculation(data,trainlabels)

In [8]:
# Separate the data into training and test sets based on the availability of labels
X_train = []
y_train = []
X_test = []
test_indices = []

for i in range(len(data)):
    if i in trainlabels:
        X_train.append(data[i])
        y_train.append(trainlabels[i])
    else:
        X_test.append(data[i])
        test_indices.append(i)

# Convert to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)

In [9]:
print("Column (Feature):", column)
print("Split value (Threshold):", split)

Column (Feature): 0
Split value (Threshold): 0.8437522590590001


### Implement a decision stump using scikit-learn library

In [10]:
# Initialize and train the DecisionTreeClassifier with max_depth=1
stump = DecisionTreeClassifier(max_depth=1, criterion='gini')
stump.fit(X_train,y_train)


DecisionTreeClassifier(max_depth=1)

In [11]:
# Make predictions on the test data
predictions = stump.predict(X_test)

# Get the column (feature) and split value used by the decision stump
column = stump.tree_.feature[0]
threshold = stump.tree_.threshold[0]

print("Column (Feature):", column)
print("Split value (Threshold):", threshold)

Column (Feature): 0
Split value (Threshold): 0.8426439762115479
